In [1]:
#Importing libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder as OHE
from sklearn.model_selection import train_test_split

/Users/nataliiashcheglova/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Loading data
data = pd.read_csv('cancelations_data_clean.csv',parse_dates=['Order Date'])
data.drop(columns=['Unnamed: 0'],inplace=True)
data.head()

,Order ID,Order Date,Fulfilment By,Sales Channel,Shipment Type,Style,SKU,Category,Size,Shipment City,...,Status Category,Price,Shipment State Matched,Free Financing,Free Shipping,Coupon,Other Promotions,No Promotions,Weekend,Stock
0,405-8078784-5731545,2022-04-30,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,mumbai,...,Cancelled,680.0,Maharashtra,False,False,False,False,True,True,32.0
1,171-9198151-1101146,2022-04-30,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,bengaluru,...,Shipped,406.0,Karnataka,True,False,False,False,False,True,96.0
2,404-0687676-7273146,2022-04-30,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,navi mumbai,...,Shipped,329.0,Maharashtra,False,True,False,False,False,True,4.0
3,403-9615377-8133951,2022-04-30,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,puducherry,...,Cancelled,791.0,Puducherry,False,False,False,False,True,True,193.0
4,407-1069790-7240320,2022-04-30,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,chennai,...,Shipped,574.0,Tamil Nadu,False,False,False,False,True,True,6.0


In [3]:
# Adding features
data['Month'] = data['Order Date'].dt.month
data['DayofMonth'] = data['Order Date'].dt.day
data['begin_of_month'] = (data['DayofMonth'] < 10).astype('uint8')
data['midddle_of_month'] = ((data['DayofMonth'] >= 10)&(data['DayofMonth'] < 20)).astype('uint8')
data['end_of_month'] = (data['DayofMonth'] >= 20).astype('uint8')
data.head()

,Order ID,Order Date,Fulfilment By,Sales Channel,Shipment Type,Style,SKU,Category,Size,Shipment City,...,Coupon,Other Promotions,No Promotions,Weekend,Stock,Month,DayofMonth,begin_of_month,midddle_of_month,end_of_month
0,405-8078784-5731545,2022-04-30,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,mumbai,...,False,False,True,True,32.0,4,30,0,0,1
1,171-9198151-1101146,2022-04-30,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,bengaluru,...,False,False,False,True,96.0,4,30,0,0,1
2,404-0687676-7273146,2022-04-30,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,navi mumbai,...,False,False,False,True,4.0,4,30,0,0,1
3,403-9615377-8133951,2022-04-30,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,puducherry,...,False,False,True,True,193.0,4,30,0,0,1
4,407-1069790-7240320,2022-04-30,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,chennai,...,False,False,True,True,6.0,4,30,0,0,1


In [4]:
# One-hot encoding categorical fetaures. I discovered in my EDA that Category, Size, SKU and State may have 
# an impact on cancelation rate. 
categorical_vars = ['SKU','Category','Size','Shipment State Matched','Shipment Type']
data_encoded = pd.get_dummies(data, columns = categorical_vars, drop_first=True)
print('The data have ', data_encoded.shape[0], ' rows and ', data_encoded.shape[1], ' columns\n')

The data have  120280  rows and  6612  columns



In [5]:
# We got too many features when one-hot encoding SKU, I'll try to limit the number of features by grouping 
# together SKUs with small number of orders.
counts = data['SKU'].value_counts()
mask = data['SKU'].isin(counts[counts<30].index)
data['SKU'][mask] = 'Other'
data['SKU'].nunique()

/var/folders/5y/xktw_n857vb915jnv6mlqv6c0000gn/T/ipykernel_7343/315692569.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['SKU'][mask] = 'Other'
/var/folders/5y/xktw_n857vb915jnv6mlqv6c0000gn/T/ipykernel_7343/315692569.py:5: Setting

1009

In [6]:
# Trying again with reduced number of SKUs
data_encoded = pd.get_dummies(data, columns = categorical_vars, drop_first=True)
print('The data have ', data_encoded.shape[0], ' rows and ', data_encoded.shape[1], ' columns\n')

The data have  120280  rows and  1085  columns



In [7]:
#Transforming target feature to numerical
data_encoded['Status Category'] = data_encoded['Status Category'].map({'Cancelled': 1, 'Shipped': 0}).values

In [8]:
# Dropping columns I won't use in my model
columns_to_drop=['Status Category','Order Date','Sales Channel','Fulfilment By','Style','Shipment City','Shipment Postal Code','Weekend']

In [9]:
# Splitting data into train & test datasets
X_train, X_test, y_train, y_test = train_test_split(data_encoded.drop(columns=columns_to_drop), 
                                                    data_encoded['Status Category'], test_size=0.3, 
                                                    random_state=47)

In [10]:
# Saving Order_ids before dropping the column
orders_list = ['Order ID']

orders_train = X_train[orders_list]
orders_test = X_test[orders_list]

X_train.drop(columns=orders_list, inplace=True)
X_test.drop(columns=orders_list, inplace=True)

X_train.shape, X_test.shape

((84196, 1076), (36084, 1076))

In [11]:
# Price and stock level don't seem to be correlated with cancellations, so I wouldn't use them in my model, 
# but for the purpose of this project I will scale them as if I'd use them.
column_names = X_train.columns

scaler = StandardScaler()
scaler.fit(X_train)

X_tr_scaled = scaler.transform(X_train)
X_tr_scaled = pd.DataFrame(X_tr_scaled, columns=column_names)

X_te_scaled = scaler.transform(X_test)
X_te_scaled = pd.DataFrame(X_te_scaled, columns=column_names)

In [12]:
# Saving pre-processed fetures in csv
X_tr_scaled.to_csv('train_features.csv',index=False)
X_te_scaled.to_csv('test_features.csv',index=False)

In [21]:
# Saving target feature in csv
y_train.to_csv('train_target.csv',index=False)
y_test.to_csv('test_target.csv',index=False)